In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools

2024-03-04 07:37:35.371364: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 07:37:35.371499: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 07:37:35.519151: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image

In [3]:
def compute_ela_cv(path, quality):
    temp_filename = 'temp_file_name.jpg'
    SCALE = 15
    orig_img = cv2.imread(path)
    orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite(temp_filename, orig_img, [cv2.IMWRITE_JPEG_QUALITY, quality])

    # read compressed image
    compressed_img = cv2.imread(temp_filename)

    # get absolute difference between img1 and img2 and multiply by scale
    diff = SCALE * cv2.absdiff(orig_img, compressed_img)
    return diff

In [4]:
image_size = (128, 128)
def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 91).resize(image_size)).flatten() / 255.0

In [5]:
X = [] # ELA converted images
Y = [] # 0 for fake, 1 for real

In [6]:
import random
path = '../input/casia-dataset/CASIA2/Au/'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png'):
            full_path = os.path.join(dirname, filename)
            X.append(prepare_image(full_path))
            Y.append(1)
            if len(Y) % 500 == 0:
                print(f'Processing {len(Y)} images')

random.shuffle(X)
X = X[:2100]
Y = Y[:2100]
print(len(X), len(Y))

Processing 500 images
Processing 1000 images
Processing 1500 images
Processing 2000 images
Processing 2500 images
Processing 3000 images
Processing 3500 images
Processing 4000 images
Processing 4500 images
Processing 5000 images
Processing 5500 images
Processing 6000 images
Processing 6500 images
Processing 7000 images
2100 2100


In [7]:
path = '../input/casia-dataset/CASIA2/Tp/'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png'):
            full_path = os.path.join(dirname, filename)
            X.append(prepare_image(full_path))
            Y.append(0)
            if len(Y) % 500 == 0:
                print(f'Processing {len(Y)} images')

print(len(X), len(Y))

Processing 2500 images
Processing 3000 images
Processing 3500 images
Processing 4000 images
4164 4164


In [8]:
X = np.array(X)
Y = to_categorical(Y, 2)
X = X.reshape(-1, 128, 128, 3)

In [9]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)
X = X.reshape(-1,1,1,1)
print(len(X_train), len(Y_train))
print(len(X_val), len(Y_val))

3331 3331
833 833


In [10]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (5, 5), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
    model.add(Conv2D(filters = 32, kernel_size = (5, 5), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))
    return model

In [11]:
model = build_model()
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 124, 124, 32)   │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 120, 120, 32)   │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 60, 60, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 60, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    29,491,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,520,034 (112.61 MB)

 Trainable params: 29,520,034 (112.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
from keras import optimizers
from keras.optimizers import Adam


model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
epochs = 24
batch_size = 32
init_lr = 1e-4
optimizer = Adam(learning_rate=init_lr)


In [13]:
early_stopping = EarlyStopping(monitor = 'val_acc',
                              min_delta = 0,
                              patience = 2,
                              verbose = 0,
                              mode = 'auto')

In [14]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Assuming you have defined and compiled your model before this point
# model = ...

x_train2 = np.array(X_train, copy=True) 
y_train2 = np.array(Y_train, copy=True) 

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    fill_mode='nearest',
    validation_split=0.2
)

datagen.fit(x_train2)  # Use x_train2 instead of X_train

# Early stopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=0, mode='min')

# Fit the model using the fit method
history = model.fit(datagen.flow(x_train2, y_train2, batch_size=32, subset='training'),
                    epochs=epochs,
                    validation_data=datagen.flow(x_train2, y_train2, batch_size=32, subset='validation'),
                    verbose=1,
                    callbacks=[early_stopping])


Epoch 1/24


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


84/84 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.7326 - loss: 3.3061 - val_accuracy: 0.8453 - val_loss: 0.4766
Epoch 2/24
84/84 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.8461 - loss: 0.3987 - val_accuracy: 0.8904 - val_loss: 0.3244
Epoch 3/24
84/84 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.8993 - loss: 0.2960 - val_accuracy: 0.8814 - val_loss: 0.3116
Epoch 4/24
84/84 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.8984 - loss: 0.2665 - val_accuracy: 0.8904 - val_loss: 0.3082
Epoch 5/24
84/84 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.9246 - loss: 0.2119 - val_accuracy: 0.9009 - val_loss: 0.3044
Epoch 6/24
84/84 ━━━━━━━━━━━━━━━━━━━━ 164s 2s/step - accuracy: 0.9381 - loss: 0.2041 - val_accuracy: 0.8994 - val_loss: 0.2783
Epoch 7/24
84/84 ━━━━━━━━━━━━━━━━━━━━ 164s 2s/step - accuracy: 0.9462 - loss: 0.1489 - val_accuracy: 0.9054 - val_loss: 0.2944
Epoch 8/24
84/84 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.9529 - loss: 0.1449 - val_accuracy: 0.8979 - val_loss: 0.

In [15]:
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [16]:
hist = model.fit(X_train,
                 Y_train,
                 batch_size = batch_size,
                 epochs = epochs,
                validation_data = (X_val, Y_val),
                callbacks = [early_stopping])

Epoch 1/24
105/105 ━━━━━━━━━━━━━━━━━━━━ 192s 2s/step - accuracy: 0.5591 - loss: 0.7323 - val_accuracy: 0.7575 - val_loss: 0.5544
Epoch 2/24
105/105 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.7283 - loss: 0.5599 - val_accuracy: 0.8403 - val_loss: 0.3981
Epoch 3/24
105/105 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.8307 - loss: 0.4044 - val_accuracy: 0.8812 - val_loss: 0.3186
Epoch 4/24
105/105 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.8675 - loss: 0.3436 - val_accuracy: 0.8908 - val_loss: 0.2868
Epoch 5/24
105/105 ━━━━━━━━━━━━━━━━━━━━ 189s 2s/step - accuracy: 0.8875 - loss: 0.2981 - val_accuracy: 0.8992 - val_loss: 0.2619
Epoch 6/24
105/105 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.8951 - loss: 0.2841 - val_accuracy: 0.9076 - val_loss: 0.2464
Epoch 7/24
105/105 ━━━━━━━━━━━━━━━━━━━━ 190s 2s/step - accuracy: 0.9079 - loss: 0.2535 - val_accuracy: 0.9088 - val_loss: 0.2372
Epoch 8/24
105/105 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.9141 - loss: 0.2440 - val_accu

In [17]:
model.save('model_casia_run1.h5')